## 2.1

1. \b\w+\b
2. \b[a-z]*b\b
3. \bb*(bab)b**\b

## 2.2

Skipping, don't need regex practice

## 2.4

To compute a cell, using the Levenshtein distance with sub-cost = 2, take the shortest distance of row above + 1, col left + 1, or diag up-left + 2. If same letter, same as 0-cost substitution

| s\t | # | d | e | a | l |
|-|-|-|-|-|-|
| # | 0 | 1 | 2 | 3 | 4 |
| l | 1 | 2 | 3 | 4 | 3 |
| e | 2 | 3 | 2 | 3 | 4 |
| d | 3 | 2 | 3 | 4 | 5 |
| a | 4 | 3 | 4 | 3 | 4 |

## 2.6

In [3]:
import numpy as np

# Cost functions
def del_cost(char):
    return 1

def ins_cost(char):
    return 1

def sub_cost(char1, char2):
    return del_cost(char1) + ins_cost(char2) if char1 != char2 else 0

# Distance function
def compute_edit_distance(source, target, del_cost=del_cost, ins_cost=ins_cost, sub_cost=sub_cost, table=False):
    n, m = len(source), len(target)
    # Initialize D, where D[i,j] is the substring at source[i] and target[j]
    # (1-indexed, since index 0 is empty string)
    D = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        D[i,0] = D[i-1,0] + 1
    for j in range(1, m+1):
        D[0,j] = D[0,j-1] + 1
        
    for i in range(1, n+1):
        for j in range(1, m+1):
            D[i,j] = min(
                # Index into source and target must be decremented bc i, j are 1-indexed
                D[i-1,j] + del_cost(source[i-1]),               # deletion
                D[i,j-1] + ins_cost(target[j-1]),               # insertion
                D[i-1,j-1] + sub_cost(source[i-1], target[j-1]) # substitution 
            )
    
    if table:
        return D
    else: 
        # D[n,m] is the final edit distance 
        return D[n,m]

print('intention => execution')
print(compute_edit_distance('intention', 'execution'))
print()
print('leda => deal')
print(compute_edit_distance('leda', 'deal', table=True))

intention => execution
8.0

leda => deal
[[0. 1. 2. 3. 4.]
 [1. 2. 3. 4. 3.]
 [2. 3. 2. 3. 4.]
 [3. 2. 3. 4. 5.]
 [4. 3. 4. 3. 4.]]


## 2.7

In [44]:
# Alignment
def compute_alignment(source, target, del_cost=del_cost, ins_cost=ins_cost, sub_cost=sub_cost, table=False):
    n, m = len(source), len(target)
    # Initialize D, where D[i,j] is the substring at source[i] and target[j]
    # (1-indexed, since index 0 is empty string)
    D = np.zeros((n+1, m+1))
    A = np.empty((n+1, m+1), dtype=(np.str_, 3))
    for i in range(1, n+1):
        D[i,0] = D[i-1,0] + 1
        A[i,0] = "↑"
    for j in range(1, m+1):
        D[0,j] = D[0,j-1] + 1
        A[0,j] = "←"
        
    # Compute distance and arrow matrices
    for i in range(1, n+1):
        for j in range(1, m+1):
            del_dist = D[i-1,j] + del_cost(source[i-1])
            ins_dist = D[i,j-1] + ins_cost(target[j-1])
            sub_dist = D[i-1,j-1] + sub_cost(source[i-1], target[j-1])
            min_dist = min(del_dist, ins_dist, sub_dist)
            
            arrows = ""
            if del_dist == min_dist:
                arrows += "↑"
            if ins_dist == min_dist:
                arrows += "←"
            if sub_dist == min_dist:
                arrows += "↖"
                
            D[i,j] = min_dist
            A[i,j] = arrows
            
    # Backtrace, return first shortest path
    source = (n,m)
    target = (0,0)
    queue = [[source]]
    seen = set()
    
    ix = 0
    path = None
    while ix < len(queue):
        path = queue[ix]
        ix += 1
        curr_node = path[-1]
        if curr_node in seen:
            continue
        seen.add(curr_node)
        if curr_node == target:
            break
        # Add children to queue
        i, j = curr_node[0], curr_node[1]
        if "↑" in A[i,j]:
            queue.append(path + [(i-1,j)])
        if "←" in A[i,j]:
            queue.append(path + [(i,j-1)])
        if "↖" in A[i,j]:
            queue.append(path + [(i-1,j-1)])
    
    if table:
        return D, A, path
    else: 
        # D[n,m] is the final edit distance 
        return D[n,m], A[n,m], path
    
print('intention => execution') 
for i in compute_alignment('intention', 'execution', table=True):
    print(i)

intention => execution
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 1.  2.  3.  4.  5.  6.  7.  6.  7.  8.]
 [ 2.  3.  4.  5.  6.  7.  8.  7.  8.  7.]
 [ 3.  4.  5.  6.  7.  8.  7.  8.  9.  8.]
 [ 4.  3.  4.  5.  6.  7.  8.  9. 10.  9.]
 [ 5.  4.  5.  6.  7.  8.  9. 10. 11. 10.]
 [ 6.  5.  6.  7.  8.  9.  8.  9. 10. 11.]
 [ 7.  6.  7.  8.  9. 10.  9.  8.  9. 10.]
 [ 8.  7.  8.  9. 10. 11. 10.  9.  8.  9.]
 [ 9.  8.  9. 10. 11. 12. 11. 10.  9.  8.]]
[['' '←' '←' '←' '←' '←' '←' '←' '←' '←']
 ['↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↖' '←' '←']
 ['↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑' '↑←↖' '↖']
 ['↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↖' '↑←' '↑←↖' '↑']
 ['↑' '↖' '←' '←↖' '←' '←' '↑←' '↑←↖' '↑←↖' '↑']
 ['↑' '↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑↖']
 ['↑' '↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↖' '←' '←' '↑←']
 ['↑' '↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑' '↖' '←' '←']
 ['↑' '↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑' '↑' '↖' '←']
 ['↑' '↑' '↑←↖' '↑←↖' '↑←↖' '↑←↖' '↑' '↑' '↑' '↖']]
[(9, 9), (8, 8), (7, 7)